# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [5]:
cur.execute('''SELECT * 
            FROM employees
            JOIN offices
            using (officeCode)''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.loc[df['city'] == 'Boston', ['lastName', 'firstName']]

,lastName,firstName
8,Firrelli,Julie
9,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [22]:
# cur.execute('''SELECT * 
#             FROM offices
#             LEFT JOIN employees
#             using (officeCode)''')
# df = pd.DataFrame(cur.fetchall())
# df.columns = [x[0] for x in cur.description]
# df.loc[df['employeeNumber'].isna()]

cur.execute('''SELECT o.officeCode, o.city, o.addressLine1, o.addressLine2, COUNT(e.employeeNumber) as numEmployees
            FROM offices o 
            LEFT JOIN employees e 
            using (officeCode)
            GROUP BY o.officeCode
            HAVING numEmployees = 0''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,officeCode,city,addressLine1,addressLine2,numEmployees
0,27,Boston,105 Cambridge Street,,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [54]:
# What are the top 3 best-selling product lines based on number of products sold?
cur.execute('''SELECT p.productLine, SUM(od.quantityOrdered) as unitsSold
            FROM orderdetails od
            LEFT JOIN products p USING (productCode)
            LEFT JOIN orders o using (orderNumber)
            GROUP BY p.productLine
            ORDER BY unitsSold DESC
            LIMIT 3''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productLine,unitsSold
0,Classic Cars,35582
1,Vintage Cars,22933
2,Motorcycles,12778


In [55]:
# Who are the top 3 selling employees based on revenue generated?
cur.execute('''SELECT e.firstName, e.lastName, SUM(pay.amount) AS revenue
            FROM customers c
            LEFT JOIN employees e ON c.salesRepEmployeeNumber = e.employeeNumber
            LEFT JOIN payments pay USING (customerNumber)
            GROUP BY e.employeeNumber
            ORDER BY revenue DESC
            LIMIT 3''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,revenue
0,Gerard,Hernandez,1112003.81
1,Leslie,Jennings,989906.55
2,Pamela,Castillo,750201.87


In [56]:
# Which office has the most sales (measured in revenue)?
cur.execute('''SELECT o.officeCode, o.city, SUM(p.amount) AS revenue
            FROM payments p 
            JOIN customers c USING (customerNumber) 
            LEFT JOIN employees e ON (c.salesRepEmployeeNumber = e.employeeNumber)
            LEFT JOIN offices o USING (officeCode)
            GROUP BY o.officeCode
            ORDER BY revenue DESC
            LIMIT 1''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,officeCode,city,revenue
0,4,Paris,2819168.9


## Level Up: Display the names of every individual product that each employee has sold

In [87]:
cur.execute('''SELECT e.employeeNumber, e.firstName, e.lastName, p.productCode, p.productName
            FROM employees e
            JOIN customers c ON (e.employeeNumber = c.salesRepEmployeeNumber)
            LEFT JOIN orders o USING (customerNumber)
            LEFT JOIN orderdetails od USING (orderNumber)
            LEFT JOIN products p USING (productCode)
            GROUP BY employeeNumber, productCode
            ORDER BY e.employeeNumber, p.productCode''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,productCode,productName
0,1165,Leslie,Jennings,S10_1678,1969 Harley Davidson Ultimate Chopper
1,1165,Leslie,Jennings,S10_1949,1952 Alpine Renault 1300
2,1165,Leslie,Jennings,S10_2016,1996 Moto Guzzi 1100i
3,1165,Leslie,Jennings,S10_4698,2003 Harley-Davidson Eagle Drag Bike
4,1165,Leslie,Jennings,S10_4757,1972 Alfa Romeo GTA
...,...,...,...,...,...
1365,1702,Martin,Gerard,S700_2834,ATA: B757-300
1366,1702,Martin,Gerard,S700_3505,The Titanic
1367,1702,Martin,Gerard,S700_3962,The Queen Mary
1368,1702,Martin,Gerard,S72_1253,Boeing X-32A JSF


## Level Up: Display the Number of Products each employee has sold

In [59]:
cur.execute('''SELECT e.employeeNumber, e.firstName, e.lastName, SUM(od.quantityOrdered) AS unitsSold
            FROM employees e
            LEFT JOIN customers c ON (e.employeeNumber = c.salesRepEmployeeNumber)
            LEFT JOIN orders o USING (customerNumber)
            LEFT JOIN orderdetails od USING (orderNumber)
            GROUP BY e.employeeNumber
            ORDER BY unitsSold DESC''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,unitsSold
0,1370,Gerard,Hernandez,14231.0
1,1165,Leslie,Jennings,11854.0
2,1401,Pamela,Castillo,9290.0
3,1501,Larry,Bott,8205.0
4,1504,Barry,Jones,7486.0
5,1323,George,Vanauf,7423.0
6,1612,Peter,Marsh,6632.0
7,1611,Andy,Fixter,6246.0
8,1337,Loui,Bondur,6186.0
9,1216,Steve,Patterson,5561.0


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!